In [1]:
%pylab inline
import altair as alt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../91-DIVOC/pages/covid-visualization/jhu-data.csv",index_col=0)

In [4]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
# Keep only data from the top 10 countries with the highest total number of Confirmed cases
top10countries = list(df.groupby("Country_Region").sum()["Confirmed"].nlargest(10).index)
df = df[df["Country_Region"].isin(top10countries)]

In [6]:
selectedRows = df[(df["Confirmed"]==0)&(df["Recovered"]==0)&(df["Active"]==0)&(df["Deaths"]==0)].index
df = df.drop(selectedRows, axis=0)

In [7]:
# df = df[df["Confirmed"]>100]

In [8]:
chart = alt.Chart(df).mark_line().encode(
    x = alt.X("Date"),
    y= alt.Y("Confirmed",aggregate="sum"),
    color = alt.Color("Country_Region")
)
# chart.interactive()
chart

alt.Chart(...)

In [9]:
# Days since 100 confirmed cases

In [10]:
firstDayWith100byCountries = df[df["Confirmed"]>100].groupby("Country_Region")["Date"].min().to_dict()

In [11]:
from datetime import datetime

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%m-%d-%Y")
    d2 = datetime.strptime(d2, "%m-%d-%Y")
    return (d2 - d1).days

In [12]:
# Testing
x = df.iloc[200,:]
print (x)
days_between(firstDayWith100byCountries[x["Country_Region"]],x["Date"])

Country_Region    United States
Province_State        Tempe, AZ
Confirmed                     1
Recovered                     0
Active                        1
Deaths                        0
Date                 02-09-2020
Name: 49, dtype: object


-24

In [13]:
firstDayWith100byCountries

{'China': '01-18-2020',
 'France': '03-01-2020',
 'Germany': '03-01-2020',
 'Iran': '02-26-2020',
 'Italy': '02-23-2020',
 'South Korea': '02-20-2020',
 'Spain': '03-04-2020',
 'Switzerland': '03-05-2020',
 'United Kingdom': '03-05-2020',
 'United States': '03-04-2020'}

In [14]:
df["Days Since First 100 Confirmed"] = df.apply(lambda x: days_between(firstDayWith100byCountries[x["Country_Region"]],x["Date"]),axis=1)

In [15]:
# df = df[df["Days Since First 100 Confirmed"]>0]

In [16]:
chart = alt.Chart(df).mark_line().encode(
    x = alt.X("Days Since First 100 Confirmed",scale=alt.Scale(domain=(0,40))),
    y= alt.Y("Confirmed",aggregate="sum",title="Total Confirmed Cases"),
    color = alt.Color("Country_Region")
)
chart.interactive()

alt.Chart(...)

In [17]:
# Logarithmic Axis
chart = alt.Chart(df).mark_line().encode(
    x = alt.X("Days Since First 100 Confirmed",scale=alt.Scale(domain=(0,40))),
    y= alt.Y("Confirmed",aggregate="sum",title="Total Confirmed Cases (Log)", scale=alt.Scale(type='log',domain=(100,100000))),
    color = alt.Color("Country_Region")
)
chart.interactive()

alt.Chart(...)

In [28]:
usa = df[df["Country_Region"]=="United States"]

In [29]:
# usa = usa.dropna()

In [30]:
top10states = list(df.groupby("Province_State").sum()["Confirmed"].nlargest(10).index)
usa = usa[usa["Province_State"].isin(top10states)]

In [31]:
firstDayWith20byState = usa[usa["Confirmed"]>20].groupby("Province_State")["Date"].min().to_dict()
usa["Days Since First 20 Confirmed"] = usa.apply(lambda x: days_between(firstDayWith20byState[x["Province_State"]],x["Date"]),axis=1)

In [32]:
chart = alt.Chart(usa).mark_line().encode(
    x = alt.X("Days Since First 20 Confirmed",scale=alt.Scale(domain=(0,24))),
    y= alt.Y("Confirmed",aggregate="sum",title="Total Confirmed Cases (Log)", scale=alt.Scale(type='log',domain=(10,50000))),
    color = alt.Color("Province_State")
)
chart.interactive()

alt.Chart(...)

#### I think there is some bug in the plot above, since it doesn't exactly reproduce [this](http://91-divoc.com/pages/covid-visualization/). It is possible that the other plot uses a different criteria for "20 cases" than ours (we look at only confirmed cases to filter), but more likely that there is a bug somewhere.